In [38]:
import pandas as pd
import numpy as np

In [39]:
tweet_data = pd.read_csv(r'C:\Users\hs414\OneDrive\Desktop\Major_2024\data\hate\users_hate_glove.content',header=None, delimiter="\t")
# all_data = pd.read_csv(r'C:\Users\hs414\OneDrive\Desktop\Major_2024\data\hate\users_hate_all.content',header=None, delimiter="\t")
# # tweet_data = tweet_data.rename(columns={0: "user_id", 301: "hate_label"})

In [40]:
tweet_data.shape

(100386, 302)

In [41]:
tweet_data = tweet_data.rename(columns={0: "user_id", 301: "hate_label"})
# all_data = all_data.rename(columns={0: "user_id", 321: "hate_label"})

In [42]:
tweet_data = tweet_data[tweet_data['hate_label'] != "other"]

tweet_data.reset_index(inplace=True)
tweet_data.shape

(4971, 303)

In [43]:
# all_data = all_data[all_data['hate_label'] != "other"]
# all_data.reset_index(inplace=True)
# all_data.shape

In [44]:
edge_data = pd.read_csv(r'C:\Users\hs414\OneDrive\Desktop\Major_2024\data\hate\users.edges', sep='\t', header=None, names=['source', 'target'])


In [45]:
edge_data['target'] = edge_data['source'].str.split(" ").str[1].astype(int)
edge_data['source'] = edge_data['source'].str.split(" ").str[0].astype(int)

In [46]:
edge_data

,source,target
0,10999,55317
1,10999,44622
2,10999,10999
3,10999,71821
4,10999,57907
...,...,...
2286587,73526,7885
2286588,73526,73526
2286589,73526,99889
2286590,73526,33299


In [47]:
user_ids = tweet_data['user_id'].unique()
cleaned_edge_data = edge_data[edge_data['source'].isin(user_ids) & edge_data['target'].isin(user_ids)]


In [56]:
cleaned_edge_data.reset_index(inplace=True)

In [57]:
tweet_data.drop(['index'],axis=1,inplace=True)

KeyError: "['index'] not found in axis"

In [58]:
tweet_data_features = tweet_data.drop(columns=['user_id','hate_label'])
tweet_data_labels = tweet_data['hate_label']

In [59]:
tweet_data_labels

0        normal
1        normal
2        normal
3        normal
4        normal
         ...   
4966     normal
4967     normal
4968     normal
4969    hateful
4970     normal
Name: hate_label, Length: 4971, dtype: object

In [60]:
tweet_data.shape

(4971, 302)

In [61]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch_geometric.data import Data
import pandas as pd
import networkx as nx

# Assuming tweet_data_labels, tweet_data_features, and tweet_graph are already defined

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(tweet_data_labels)
edge_index = cleaned_edge_data[['source', 'target']].values.T

graph = Data(edge_index=torch.tensor(edge_index, dtype=torch.long))
features_array = tweet_data_features.values

x = torch.tensor(features_array, dtype=torch.float)
y = torch.tensor(labels_encoded, dtype=torch.long)
data = Data(x=x, edge_index=graph.edge_index, y=y)
data


Data(x=[4971, 300], edge_index=[2, 15141], y=[4971])

In [62]:
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

# Define GCN model
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Instantiate GCN model
model = GCN(input_dim=tweet_data_features.shape[1], hidden_dim=64, output_dim=2)  # Assuming features is your feature tensor

# Define loss function and optimizer
criterion = torch.nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(data)
    loss = criterion(outputs, data.y)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

# Evaluation
model.eval()
with torch.no_grad():
    outputs = model(data)
    _, predicted = torch.max(outputs, 1)
    accuracy = torch.sum(predicted == data.y).item() / len(data.y)
    print(f'Test Accuracy: {accuracy}')


RuntimeError: index 15570 is out of bounds for dimension 0 with size 4971

In [ ]:
15570 in x

True